# Handling languages
In this section we want to address the issue of languages wince we do not have only english in our dataset.

In [2]:
import pandas as pd
import sqlite3
from textblob import TextBlob

Let's first take the lyrics with their words from our database:

In [3]:
conn = sqlite3.connect("../datasets/mxm_dataset.db")

In [4]:
cursor = conn.cursor()
cursor.execute("SELECT track_id, word, count FROM lyrics ORDER BY track_id;")
track_word_count = cursor.fetchall()

In [5]:
sqldb_frame = pd.DataFrame(track_word_count, columns=["track_id", "word", "count"])
del track_word_count
sqldb_frame["word"]=sqldb_frame["word"].astype(str)

In [6]:
lyrics_words = sqldb_frame.set_index("track_id")

In [174]:
test_dataframe = pd.DataFrame(data={"word": ["machin", "hello", "morning", "soir", "hello"]})
test_dataframe.word.apply(lambda x : is_english(x)).mode()

0    True
dtype: bool

We will use [pyenchant](http://pythonhosted.org/pyenchant/) to make sure our words belong to the english US dictionary. The method we use is simple: if most of the words belong to the english dictionary (more than half) then we consider the song as english. The reason we don't say that all the words should be english is because we have a lot of mispelling in the lyrics dataset. Take for example:

In [37]:
lyrics_words.loc["TRAADKW128E079503A"].sort_values(by="count").head(10)

,word,count,is_english
track_id,,,
TRAADKW128E079503A,someth,1,False
TRAADKW128E079503A,money,1,True
TRAADKW128E079503A,ani,1,False
TRAADKW128E079503A,late,1,True
TRAADKW128E079503A,ai,1,False
TRAADKW128E079503A,save,1,True
TRAADKW128E079503A,lose,1,True
TRAADKW128E079503A,mean,1,True
TRAADKW128E079503A,myself,1,True


In [169]:
import enchant
d = enchant.Dict("en_US")
def is_english(x):
    return d.check(x)
track_sent = track_frame.drop("count", 1)
track_sent.word.apply(lambda x : is_english(x))

0    True
dtype: bool

In [168]:
track_sent.mode()

,word
0,a
1,and
2,are
3,for
4,got
5,have
6,i
7,is
8,it
9,me


In [85]:
def detect_language(word):
    if len(word) >= 3:
        return TextBlob(word).detect_language()
    else :
        return None
detect(track_frame.word.str.cat(sep=' '))

'en'

In [64]:
for i in range(30):
    b = TextBlob("lifeless")
    print(b.detect_language())

en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en
en


In [28]:
for track_id in lyric_words.index.unique():
    track_frame = lyrics_words.loc[track_id]
    

TRAAAAV128F421A322
TRAAABD128F429CF47
TRAAAED128E0783FAB
TRAAAEF128F4273421
TRAAAEW128F42930C0
TRAAAFD128F92F423A
TRAAAGF12903CEC202
TRAAAHJ128F931194C
TRAAAHZ128E0799171
TRAAAJG128F9308A25
TRAAAOF128F429C156
TRAAARJ128F9320760
TRAAAUC128F428716F
TRAAAZF12903CCCF6B
TRAABEV12903CC53A4
TRAABHB12903CAFC2F
TRAABHC128F933A3F8
TRAABIG128F9356C56
TRAABJS128F9325C99
TRAABJV128F1460C49
TRAABLR128F423B7E3
TRAABOA128F933684A
TRAABOG128F42955B1
TRAABPG128F14774DD
TRAABRX12903CC4816
TRAABVM128F92CA9DC
TRAABXH128F42955D6
TRAACER128F4290F96
TRAACFV128F935E50B
TRAACHN128F1489601
TRAACIE128F428495B
TRAACIR128F42963AC
TRAACJC128F934ABB5
TRAACPH12903CF5F14
TRAACQW128F428854F
TRAACRY12903CAF2C2
TRAACUP128E0789C69
TRAACZN128F93236B1
TRAADAA128F92F7043
TRAADBN128F932D00A
TRAADCQ128F93436C3
TRAADFO128F92E1E91
TRAADKA12903CD2511
TRAADKW128E079503A
TRAADKZ128F149BDFF
TRAADLH12903CA70EE
TRAADNA128F9331246
TRAADNL128F14519DF
TRAADNN128F42B1D94
TRAADQL128F427D281
TRAADQW128F427CE68
TRAADQX128F422B4CF
TRAADRX12903

TRAFDQF12903CB5F7B
TRAFDTD128F425F22C
TRAFDXG128F4255282
TRAFDYB128F427D7A0
TRAFDZB128F4257486
TRAFDZB128F42868AA
TRAFDZJ128F93021B8
TRAFEEO12903CFEC87
TRAFEFH128F427F2FA
TRAFEGG128F42675F1
TRAFEJW128F423980F
TRAFEML128F1497485
TRAFENJ128F427738F
TRAFEPC128F4237505
TRAFEPZ128F42902A8
TRAFERD128F428B76A
TRAFETM128F149C4E6
TRAFEVM128F42278B0
TRAFEVZ128F9340E0C
TRAFEWL128F9345C1E
TRAFEYZ128F427C38C
TRAFFAF128F14979AE
TRAFFCX128E0783E62
TRAFFGI128F9342668
TRAFFGO12903CB78B0
TRAFFJV128F92D6FB5
TRAFFKC128F931CA02
TRAFFRS128F421C84A
TRAFFUU128E0788649
TRAFFVN128F4288E46
TRAFFVO128F425F27E
TRAFFYP128F92E4274
TRAFGCS128F4270F65
TRAFGEV128F9326861
TRAFGKK128F4282267
TRAFGKU128F42889EF
TRAFGLI128F14ADAF4
TRAFGLZ128F42864AF
TRAFGMJ128F930F532
TRAFGMQ128F92FE322
TRAFGMY128F934DB62
TRAFGRU128F428D900
TRAFGST128F92E9390
TRAFGTI12903CD4576
TRAFGUG12903CF2160
TRAFGUI128F93298D9
TRAFGXU128F427B862
TRAFGYR128F930F410
TRAFHAN128F42584FD
TRAFHEV128F4260C6F
TRAFHJI128F9302CFF
TRAFHKA128F429F7C0
TRAFHLO128F1

TRAKJKM128F42794AD
TRAKJKQ128F42A2E33
TRAKJNE128F149962E
TRAKJQH128F423D6A6
TRAKJTB128F148C560
TRAKJUF128F92F4461
TRAKJXJ128F9319BBA
TRAKJYQ128F934B22E
TRAKJYW12903CF5E95
TRAKKAL12903CB0F12
TRAKKAO12903D05592
TRAKKBP12903CF7283
TRAKKCB128F423C5C7
TRAKKCD128F935A563
TRAKKDV128F4288433
TRAKKEV128F931F80B
TRAKKEW128F428B9D3
TRAKKFK128F14A895A
TRAKKFL128EF34F455
TRAKKOH128F42AF40E
TRAKKPO128F145C3AC
TRAKKSI128F1465C5A
TRAKKST128F148C5A8
TRAKKTA128F428A2F1
TRAKKTI128F42B4F25
TRAKKTT128F931897F
TRAKKVD128F14A895B
TRAKKVK128F42454A5
TRAKKWK128F425F3E7
TRAKLDC128F429359D
TRAKLEZ128F4270649
TRAKLGR12903CE559B
TRAKLHO128F42BA554
TRAKLHO128F9359E2B
TRAKLIB128F427019D
TRAKLML128E07999DA
TRAKLNJ128F93196B9
TRAKLOO128F42AE70E
TRAKLQC12903CE09FD
TRAKLQP128F932A72C
TRAKLSA128F42B3009
TRAKLSB128F428DFC8
TRAKLUK128F1491227
TRAKLUM128F428C6CB
TRAKLWA128F4296E3B
TRAKLXJ128F42B4FD1
TRAKLXT128F92E388A
TRAKLYK128F92FD4DC
TRAKMES128F42697C9
TRAKMGY128F4269E37
TRAKMHT128E0788B0C
TRAKMIO128F425E76B
TRAKMJH128F4

TRAPGOC128F932F01A
TRAPGOZ128F933A945
TRAPGSJ12903CB2068
TRAPGSN12903D02BF8
TRAPGWP12903CA9CEF
TRAPGXA128F428CE61
TRAPGYG128F9320B94
TRAPGYN128F92C40CC
TRAPGZO128F4294D38
TRAPHAH128F42677A4
TRAPHAR128F428F295
TRAPHAX128F42764FE
TRAPHDI128F92F5EA1
TRAPHGM128F4278969
TRAPHQW128F92FE58E
TRAPHRO128E0792510
TRAPHSY128F4266893
TRAPHTA12903CBEBAD
TRAPHTQ128F9310DF0
TRAPHTT128F92C7557
TRAPHUO12903CC2719
TRAPHYD128F931CD52
TRAPHYP128F9341FE1
TRAPHZT128F147A02B
TRAPIFT128F42948AD
TRAPIIP128F426939A
TRAPIIR128F42AEE95
TRAPIJO128EF3681CA
TRAPILA128F92F9C73
TRAPILO12903CEA85D
TRAPIPG128F42521C6
TRAPIPT128F4288760
TRAPIRC12903CC50A6
TRAPISC128F14671D3
TRAPISE12903CDF2B0
TRAPISF128F42383AE
TRAPJAM128F934B7DB
TRAPJDV128EF364194
TRAPJEB12903CC8818
TRAPJFL128F42AA7B4
TRAPJGX128F92F43A7
TRAPJHO128F93418A7
TRAPJIN128F425E152
TRAPJLL128F92EC0E1
TRAPJLY128F4287BC1
TRAPJMS128F9327745
TRAPJNC128F92EF098
TRAPJNC12903CEB4E2
TRAPJPD128F14572B3
TRAPJPU128F1492BB1
TRAPJTF128F428ED70
TRAPJUS128F9342101
TRAPJZC128F4

TRAULAS12903CF445B
TRAULEG128F92F035B
TRAULFN128F421B980
TRAULGT12903D10B94
TRAULGY128F14B03E0
TRAULJK128F4294B58
TRAULLZ128F426892B
TRAULNQ128F42A8E17
TRAULOF128F931C6DB
TRAULOY12903C9961F
TRAULPJ128F92F1EE8
TRAULQH128F4296B5E
TRAULTM128F933D636
TRAULXY128F1452F8E
TRAULZJ128F148B2CA
TRAUMBP12903D019C6
TRAUMEY128F4289C47
TRAUMGB128F427597A
TRAUMIX12903CE640C
TRAUMMP12903C9D5C4
TRAUMNH128F92FB67F
TRAUMRF128F93066BE
TRAUMTC128F14594D2
TRAUMVS128F42922A8
TRAUMWA128F427B071
TRAUMYC128F92D46B3
TRAUNCN128F9320F26
TRAUNDI128F4256912
TRAUNDW128F4266AFA
TRAUNEL128F1467D5C
TRAUNFE128F4258FE4
TRAUNFF128F92F059E
TRAUNGB128F933E4B6
TRAUNHR128F92DBBC1
TRAUNJW128F4267AEF
TRAUNMM128F149C4E2
TRAUNNH12903CB038C
TRAUNPF128F930B1D6
TRAUNPM12903CB7CCC
TRAUNPS128F146B31C
TRAUNPY128EF340469
TRAUNQT128F426DDF5
TRAUNRN128F9311084
TRAUNSA128EF3443D4
TRAUNSC128F930C8D0
TRAUNTO128F932A772
TRAUNWD128F14703EA
TRAUNWP128F146E6D8
TRAUOAA128F4245ECF
TRAUOBI12903CD9E09
TRAUOCW128F425AE4A
TRAUODU128F9307467
TRAUOFO128F9

...

TRFVKIV128E079268D
TRFVKIZ128F425ACA3
TRFVKJM128F4297716
TRFVKNJ128F425D347
TRFVKPS12903CDF89D
TRFVKRA128F92F4E17
TRFVKTX128F9308613
TRFVKVH128E078A1B1
TRFVKVU128F930BFA0
TRFVKWP128F1458DC9
TRFVKXJ128F146228F
TRFVKYF128F92D8237
TRFVKYH128F422DE79
TRFVLAC128F930F71A
TRFVLDF128E07823E1
TRFVLDM128F92EE889
TRFVLDP12903CB6AF6
TRFVLFA128F42328B2
TRFVLGP128F93407EC
TRFVLHD128F429C910
TRFVLHQ128F932E729
TRFVLJX128F92DE3DD
TRFVLOB128F92F691B
TRFVLOW12903CBF07E
TRFVLPG128F42726F3
TRFVLSZ128F429984D
TRFVMAU128F4273D80
TRFVMCB128E0798B96
TRFVMDX128F933F136
TRFVMEH12903CBC0A1
TRFVMKU128F92E27EA
TRFVMMN128F424975A
TRFVMMP128F42AD9A9
TRFVMMY128F14921CC
TRFVMNO128F421A87B
TRFVMNR128F4257F93
TRFVMOQ12903CF294C
TRFVMQS128EF3493FC
TRFVMQS128F146625D
TRFVMUS128F92F1423
TRFVMYT128E0796A41
TRFVMZY128F427189B
TRFVNAL128F4257F6B
TRFVNAT128F4263C2C
TRFVNBN12903CFE37A
TRFVNBO128F428C33A
TRFVNCR128F934339D
TRFVNFD12903CC1884
TRFVNPM128F9346540
TRFVNST128F42A75C9
TRFVNVD128EF357624
TRFVNWD128F4270C74
TRFVNWU12903

TRGAPMD128F423ED6B
TRGAPNQ128F429B5BC
TRGAPOC128F1488FDB
TRGAPPB128F425A012
TRGAPQX128F9334863
TRGAPSK128F145E846
TRGAPST128F428076D
TRGAPUX128F9350366
TRGAPWJ128F1455E36
TRGAPXB128F1484EB7
TRGAPXK128F92FC374
TRGAQBF128F426B35A
TRGAQDP128F427EEFF
TRGAQDY12903D02133
TRGAQGF128F42A4C32
TRGAQGW128F42287EF
TRGAQHA12903CE4C1C
TRGAQHP128F9339692
TRGAQLC128F148CAB2
TRGAQOP12903D0844F
TRGAQPE12903CF09F0
TRGAQQY128F4272FAE
TRGAQRS128F421536A
TRGAQUZ128F42644D4
TRGARBN128F9330B55
TRGARDI12903CC430E
TRGARFD128F92E092A
TRGARGA128F93594D3
TRGARGI128F426AD21
TRGARHX128F92DFCA3
TRGARLY128F145898E
TRGARMG128F93225D8
TRGARND128F9302BE6
TRGARNK128F1490840
TRGAROR128F4238D48
TRGARWB128F92EF574
TRGASAI128F426782E
TRGASHG12903CC0BB7
TRGASHP128F9314EBB
TRGASIV128F429230D
TRGASJP128F4264794
TRGASKC128F42310A7
TRGASKI128F934B4CA
TRGASNC128F42639B7
TRGASOO128F4278613
TRGASPR128F93322E7
TRGASRW128F934AAFF
TRGASSQ128F42815B6
TRGASYC128F4263213
TRGASYV128F426C3F6
TRGASZG128F425C688
TRGATAP128F4260DA0
TRGATAZ128E0

TRGGSZH128F9330DAA
TRGGSZW128E0785A49
TRGGTAJ128F932610B
TRGGTCW128F934C513
TRGGTDM128F148C7EF
TRGGTEM128F42963A7
TRGGTFM128F428AE0E
TRGGTGY128F4265D0F
TRGGTOH128F149C288
TRGGTOM12903CAC8C4
TRGGTRU128F42A9504
TRGGTWX128F9343DE3
TRGGTXA128F429488F
TRGGTZI128F933FB50
TRGGTZZ128F9300B54
TRGGUAV128F930690B
TRGGUBO128F4269169
TRGGUBS128F14A14E3
TRGGUIZ128F42AB402
TRGGUKG12903CC47E9
TRGGUKJ12903CC4F5C
TRGGULV128F4295F6A
TRGGUSX128F42930B2
TRGGUVE128F42A8AA8
TRGGUZB128F4277F00
TRGGVBU128F148D609
TRGGVCH128F9315C8B
TRGGVDT128F933B77C
TRGGVHL128F429B3CD
TRGGVHO128F930B2E9
TRGGVHT128F4292016
TRGGVLC128F9320F31
TRGGVLW128EF34AC28
TRGGVLY128F93289FD
TRGGVNE128EF348615
TRGGVOW12903D081F5
TRGGVRR128F932A54A
TRGGVRT12903CB6E4A
TRGGVSI128F4285378
TRGGVYK128F4227075
TRGGWDJ128F42950FB
TRGGWEE128F92D5BCA
TRGGWFP128F92D289C
TRGGWGB128F9342644
TRGGWGG128F42684C0
TRGGWHU128F426FEE5
TRGGWKE128F4218797
TRGGWLF128F4277311
TRGGWLN128F4256EBF
TRGGWQU128F428B23D
TRGGWWG128F4263061
TRGGWWJ128F9341CA3
TRGGWYE12903

TRGMTGO128F422DA6D
TRGMTHD128F931FC41
TRGMTNW128F427F166
TRGMTSY12903CAE3D8
TRGMTWH128F424C660
TRGMTXH128F428B70B
TRGMTZN128F42590B4
TRGMUAF12903CAB120
TRGMUAY128F9344366
TRGMUBC128F14862A6
TRGMUCD128F145260F
TRGMUEF128F423CFF0
TRGMUEI128F1451BA4
TRGMUEV128F426450E
TRGMUGH128F145F98E
TRGMUKE128F931413E
TRGMULK128F934B5B9
TRGMULQ128F1466D7F
TRGMUNB128F42329DD
TRGMUOJ128F9330D45
TRGMUPH128F422B2A8
TRGMUZB128F42318FA
TRGMUZF128F4248182
TRGMVBO128F425F0B6
TRGMVBP128F14A8F4C
TRGMVDA12903CD02EF
TRGMVEY128EF343A32
TRGMVGH128F428BC41
TRGMVHN128E078B0F7
TRGMVJU128F422C14B
TRGMVJV12903CAF427
TRGMVNM128F147922B
TRGMVUU128F932D188
TRGMVWD12903CA8B08
TRGMVXU128E07818F2
TRGMVZO128E078607E
TRGMWAZ128F1481494
TRGMWCB128F4279AF5
TRGMWEN128F92D5CCB
TRGMWFN128F42A42A3
TRGMWFX128F4242520
TRGMWGB128F147C328
TRGMWIF128F930364D
TRGMWJZ128F427A720
TRGMWKU128F4259031
TRGMWMM128F9323789
TRGMWNH128F42628F4
TRGMWQK128E0791772
TRGMWRR12903CFE2F5
TRGMWRY128F9359313
TRGMWSV128F42AF59E
TRGMWUB128F428DBE8
TRGMWXD128F9

KeyboardInterrupt: 

In [9]:
import enchant
d = enchant.Dict("en_US")
def is_english(word):
    return d.check(word)
lyrics_words["is_english"] = lyrics_words.word.apply(lambda word: is_english(word))

In [189]:
lyrics_is_english = lyrics_words.is_english.groupby(lyrics_words.index)

In [14]:
lyrics_is_english = lyrics_words.is_english.groupby(lyrics_words.index).agg(lambda x : x.value_counts().index[0])

In [31]:
tracks_english = lyrics_is_english[lyrics_is_english==True].reset_index().track_id

In [32]:
tracks_english.to_csv(r'../datasets/tracks_english.csv')

In [33]:
lyrics_words.loc["TRAADKW128E079503A"]

,word,count,is_english
track_id,,,
TRAADKW128E079503A,i,37,True
TRAADKW128E079503A,the,9,True
TRAADKW128E079503A,you,15,True
TRAADKW128E079503A,to,10,True
TRAADKW128E079503A,and,21,True
TRAADKW128E079503A,a,23,True
TRAADKW128E079503A,me,2,True
TRAADKW128E079503A,it,5,True
TRAADKW128E079503A,not,3,True
